In [3]:
import os
from langchain.llms.openai import OpenAI

In [2]:
os.environ["OPENAI_API_KEY"] = '****************************************'

In [6]:
llm = OpenAI(temperature=0.6)

In [7]:
text = "What is the capital of India"
print(llm.predict(text))



The capital of India is New Delhi. 


### Huggingface

In [20]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]='**********************************'

In [21]:
from langchain import HuggingFaceHub
llm_hug_face = HuggingFaceHub(repo_id = "google/flan-t5-base", model_kwargs = {"temperature":0, 
                                                                               "max_length":64})

In [26]:
output = llm_hug_face.predict("Can you tell me the capital of USA")

In [27]:
print(output)

Washington, D.C.


### Prompt templates

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [37]:
prompt_template = PromptTemplate(input_variables=['country'], 
                                 template = "Tell me the capital of this {country}")
prompt_template.format(country = 'India')

'Tell me the capital of this India'

In [38]:
chain = LLMChain(llm = llm, prompt=prompt_template)
chain.run('india')

'\n\nThe capital of India is New Delhi.'

### Combining Multiple Chains Using simple Sequential Chain

In [42]:
from langchain.chains import SimpleSequentialChain

In [43]:
capital_prompt = PromptTemplate(input_variables=['country'], 
                               template="Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt = capital_prompt)

In [44]:
famous_template = PromptTemplate(input_variables=['capital'], 
                                template = "Suggest me some amazing places to visit in {capital}")

In [45]:
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [47]:
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
print(chain.run("India"))

 Here are some amazing places to visit in Delhi:

1. Red Fort - This iconic monument is a must-visit for its stunning architecture and historical significance.

2. Qutub Minar - Another famous monument, this 73-meter tall tower is a UNESCO World Heritage Site and a beautiful example of Indo-Islamic architecture.

3. India Gate - This war memorial is a popular spot for picnics and a great place to soak in the city's atmosphere.

4. Humayun's Tomb - This magnificent tomb is the final resting place of Mughal Emperor Humayun and is known for its beautiful gardens and intricate architecture.

5. Lotus Temple - This Bahá'í House of Worship is a peaceful and serene place to visit, with its lotus-shaped structure and tranquil atmosphere.

6. Akshardham Temple - This massive Hindu temple complex is a modern architectural marvel and a popular spot for spiritual seekers.

7. Chandni Chowk - This bustling market in Old Delhi is a shopper's paradise, offering everything from traditional Indian hand

### Sequential Chain

In [50]:
from langchain.chains import SequentialChain

In [51]:
capital_prompt = PromptTemplate(input_variables=['country'], 
                               template="Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt = capital_prompt, output_key='capital')

In [52]:
famous_template = PromptTemplate(input_variables=['capital'], 
                                template = "Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key='places')

In [53]:
chain = SequentialChain(chains = [capital_chain, famous_chain], input_variables=['country'], 
                       output_variables=['capital', 'places'])

In [55]:
print(chain({'country':'India'}))

{'country': 'India', 'capital': '\n\nThe capital of India is New Delhi.', 'places': " It is a bustling city with a rich history and culture. Here are some amazing places to visit in New Delhi:\n\n1. Red Fort - This iconic red sandstone fort was built by the Mughal emperor Shah Jahan and is a UNESCO World Heritage Site. It is a magnificent example of Mughal architecture and houses several museums and galleries.\n\n2. India Gate - A war memorial dedicated to the Indian soldiers who lost their lives during World War I, India Gate is an important landmark in New Delhi. It is a popular spot for picnics and evening walks.\n\n3. Qutub Minar - Another UNESCO World Heritage Site, Qutub Minar is a 73-meter tall tower built by Qutub-ud-din Aibak in the 12th century. It is surrounded by several other historical structures and is a must-visit for history buffs.\n\n4. Lotus Temple - This beautiful temple is shaped like a lotus flower and is a symbol of unity and peace. It is open to people of all fa

### Chatmodels with ChatOpenAI

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [60]:
chatllm = ChatOpenAI(temperature=0.6, model='gpt-3.5-turbo')

In [61]:
chatllm([SystemMessage(content="You are a comedian AI assistant"),
         HumanMessage(content="Please provide some comedy punchlines on AI")
        ])

C:\Users\n.anurag\AppData\Local\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='1. "Why did the AI break up with its computer girlfriend? She kept trying to Ctrl-Alt-Delete their relationship!"\n2. "I asked my AI assistant to tell me a joke, and it replied, \'I would, but I\'m afraid it might crash and burn!\'"\n3. "I told my AI to make me a sandwich, and it replied, \'Sorry, I\'m more of a \'byte\' than a \'bite\' kind of assistant!\'"\n4. "Why did the AI go to therapy? It had too many unresolved loops and couldn\'t function properly!"\n5. "I tried to teach my AI how to dance, but all it did was the robot. Literally, it just stood there and did the robot dance!"')

### Prompt Template+LLM+Output Parsers

In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [63]:
class CommaSeperatedOutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [64]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([("system",template),("human",human_template)])

In [65]:
chain = chatprompt|chatllm|CommaSeperatedOutput()

In [66]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' bright', ' astute', ' sharp']

In [67]:
chain.invoke({"text":"clever"})

['smart', ' intelligent', ' astute', ' sharp', ' quick-witted']

In [68]:
chain.invoke({"text":"beautiful"})

['lovely', ' attractive', ' stunning', ' charming', ' gorgeous']